In [14]:
from __future__ import print_function
import sys
print (sys.executable)

/home/ubuntu/anaconda3/envs/tensorflow_p36/bin/python


In [16]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

import data_batch as data

#### Define the blocks
Always remember the mental model:
input => representation learning => loss => optimizer

Step 1: define input (and output, because during the training we need the ground truth as reference)

In [17]:
tf.reset_default_graph()
# [1] Input
x = tf.placeholder(tf.float32, [None, data.INTERVAL_SIZE])
# y_ holds the true output, 
# we use this placeholder to pass the true recognition result to teach the network
y_ = tf.placeholder(tf.int32, [None])

Step 2: Representation learning (the network that reveals the information contained in the image). Here we use 1 layer of fully connected network

In [18]:
# [2] network, first layer
weight1 = tf.Variable(tf.random_normal([data.INTERVAL_SIZE, 76])) # can use 76 for simplicity
bias1 = tf.Variable(tf.random_normal([76]))
act_hidden1 = tf.nn.relu(tf.add(tf.matmul(x, weight1), bias1))

# Network 2nd layer
# weight2 = tf.Variable(tf.random_normal([512, 512])) # can use 76 for simplicity
# bias2 = tf.Variable(tf.random_normal([512]))
# act_hidden2 = tf.nn.relu(tf.add(tf.matmul(act_hidden1, weight2), bias2))

# # Network 3rd layer
# weight3 = tf.Variable(tf.random_normal([152, 152])) # can use 76 for simplicity
# bias3 = tf.Variable(tf.random_normal([152]))
# act_hidden3 = tf.nn.relu(tf.add(tf.matmul(act_hidden2, weight3), bias3))

# # Network 4th layer
# weight4 = tf.Variable(tf.random_normal([152, 76])) # can use 76 for simplicity
# bias4 = tf.Variable(tf.random_normal([76]))
# act_hidden4 = tf.nn.relu(tf.add(tf.matmul(act_hidden3, weight4), bias4))

# Output layer
weight_output = tf.Variable(tf.random_normal([76, 76])) # 76 x 76 
bias_output = tf.Variable(tf.random_normal([76]))
output = tf.add(tf.matmul(act_hidden1, weight_output), bias_output)

# while the layer is not the last layer, we can have any number, bottle neck layer < 76, extract information > 76

Step 3: loss function, to define what's a good prediction 

In [19]:
# [3] loss, this loss function below outputs a scalar to show who close the network's guess
# O can represent the true recognition result y_. 
# Here O[j] shows a rating of how likely x is digit j, we will show that later
loss_all = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y_)
loss = tf.reduce_mean(loss_all)

Step 4: Optimizer, strategy for training

In [20]:
# [4] Optimizer
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)
# [optional, compute accuracy so that we can know how well we are doing while training]
prediction = tf.argmax(tf.nn.softmax(output), 1, output_type=tf.int32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, y_), tf.float32))

Scalar summaries (graphs)

In [22]:
with tf.name_scope("summaries"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged = tf.summary.merge_all()
    
train_writer = tf.summary.FileWriter('train', sess.graph)
#test_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/test')

#### Training
Call "train_step" a number of time; you can call this block multiple times until it reaches a good accuracy

In [24]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Randomly draw a file, for that file, randomly draw an interval of 1024 and find corresponding label
# Training, stop when objective value (o) is small enough
for step in range(200):
    batch_xs, batch_ys = data.next_train_batch(500)
    _, summary, v_loss = sess.run([train_step, merged, loss], 
                                  feed_dict={x: batch_xs, y_: batch_ys})
    train_writer.add_summary(summary, step)
    if (step % 100 == 99):
        batch_xs, batch_ys = data.next_test_batch(1000)
        v_acc, v_loss = sess.run([accuracy, loss], feed_dict={x: batch_xs, y_: batch_ys})
        # print the loss every 100 interations
        print('accuracy=', v_acc, 'loss=', v_loss)

accuracy= 0.023 loss= 48.5586
accuracy= 0.025 loss= 36.731
